In [1]:
import json
import re
import os
import gzip
import numbers
import numpy as np
import pandas as pd
import platform
import multiprocessing as mp
from functools import partial
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
from IPython.display import display, HTML
from datetime import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime as dt, timedelta as td

spectra_map = {0: 'M', 1: 'K', 2: 'G', 3: 'F', 4: 'A', 5: 'B', 6: 'O', 7: 'X'}
singularity_flags = {
    'TidalLocked': 1,
    'TidalLocked2': 2,
    'TidalLocked4': 4,
    'LaySide': 8,
    'ClockwiseRotate': 16,
    'MultipleSatellites': 32,
}
star_type_map = {
    0: 'Main Sequence',
    1: 'Giant',
    2: 'White Dwarf',
    3: 'Neutron Star',
    4: 'Black Hole',
}
vein_type_map = {'verbose': {
    'Iron': 'Iron', 'Copper': 'Copper', 'Silicium': 'Silicon', 'Titanium': 'Titanium', 
    'Stone': 'Stone', 'Coal': 'Coal', 'Oil' : 'Oil', 'Fireice': 'Fire ice', 'Diamond': 'Kimberlite',
    'Fractal': 'Fractal Silicon', 'Crysrub': 'Organic Crystal', 'Grat': 'Optical Grating', 
    'Bamboo': 'Spiniform Stalagmite Crystal', 'Mag': 'Unipolar Magnets', },
                 'terse': {
    'Iron': 'Iron', 'Copper': 'Copper', 'Silicium': 'Silicon', 'Titanium': 'Titanium', 
    'Stone': 'Stone', 'Coal': 'Coal', 'Oil' : 'Oil', 'Fireice': 'Fire ice', 'Diamond': 'Kimberlite',
    'Fractal': 'Fractal Silicon', 'Crysrub': 'Organic Crystal', 'Grat': 'Optical Grating', 
    'Bamboo': 'Spin. Stal. Crystal', 'Mag': 'Uni. Magnets', }
}['terse']
vein_types = list(vein_type_map.values())
ocean_types = ['Water', 'Sulfuric acid']
base_types = ['Iron', 'Copper', 'Silicon', 'Titanium', 'Stone', 'Coal']
gas_types = ['Hydrogen gas', 'Deuterium gas', 'Fire ice gas',]
S_scaled = gas_types + ['Oil']
M_scaled = [x for x in vein_types if x not in S_scaled]

previous_runs = [
    84459701, 16015752, 22778246, 26027001, 57786559, 51279269, 94520636, 
    32680784, 88834638, 22161804, 87759803, 76768927, 18238299, 43378616,
    84459702, 91684768,
]

%load_ext autotime

time: 155 µs (started: 2021-10-29 08:03:15 +01:00)


In [2]:
export_path = {
    # original Windows location
    'Chonyi': '/mnt/c/' + r'\Users\Drake\Documents\Dyson Sphere Program\dsp_seedexporter'.replace('\\', '/'),
    
    # NTFS mount in Linux
    'chonyi': '/media/drake/Windows/Users/Drake/Documents/Dyson Sphere Program/dsp_seedexporter/',
    
    # copied from Chonyi
    'blincoln': os.path.expanduser('~/data/dsp/seeds'),
    'megalith': os.path.expanduser('~/data/dsp/seeds'),
}[platform.node()]
seed_re = re.compile(r'seed_(?P<seed>\d{8}).json.gz')

time: 435 µs (started: 2021-10-29 08:03:15 +01:00)


In [3]:
files = sorted(filter(seed_re.match, os.listdir(export_path)))
seeds = list(map(lambda x: seed_re.match(x).group('seed'), files))
print('{:d} files detected'.format(len(files)))

9352 files detected
time: 118 ms (started: 2021-10-29 08:03:15 +01:00)


In [4]:
def process_universe(func, filename):
    file = os.path.join(export_path, filename)
    try:
        with gzip.GzipFile(file, mode='r') as fd:
            universe = json.load(fd)
            universe['meta']['mtime'] = dt.fromtimestamp(os.stat(file).st_mtime)
    except EOFError as e:
        raise ValueError('Bad file: ' + file)
    return func(universe)

def passthrough(x):
    return x

subset = ['seed_18238299.json.gz', 'seed_94520636.json.gz', 'seed_94520637.json.gz']
with mp.Pool(mp.cpu_count() - 1) as pool:
    universes = pool.map(partial(process_universe, passthrough), subset, chunksize=100)
universe = universes[0]

time: 79.5 ms (started: 2021-10-29 08:03:15 +01:00)


In [5]:
def distance(star):
    return sum(np.fromiter(star['position'].values(), dtype='float') ** 2) ** 0.5

distance_df = pd.DataFrame.from_records(
    data=[(star_name, distance(star)) for star_name, star in universe['star'].items()], 
    columns=['star_name', 'abs. dist.']
).sort_values('abs. dist.')
distance_df

,star_name,abs. dist.
0,Procyon,0.000000
1,Tramontana,2.960353
16,Zeta Arietis,4.005012
17,27 Ophiuchii,4.611032
4,Lambda Sagittae,5.189216
...,...,...
40,ζ Chamaeleontis,27.360977
45,Cymbae,30.245616
51,Theta Puppis,33.166251
60,Arided,36.442384


time: 12.7 ms (started: 2021-10-29 08:03:15 +01:00)


In [6]:
home_star = distance_df.iloc[0]
assert home_star['abs. dist.'] == 0.
home_system = universe['star'][home_star['star_name']]

time: 8.65 ms (started: 2021-10-29 08:03:15 +01:00)


In [7]:
def map_strings(universe):
    planet_types = []
    star_types = []
    singularities = []
    planet_count = 0
    for star in universe['star'].values():
        try:
            star_types.append((star['type'], star['typeString']))
        except KeyError:
            pass
        for planet in star['planet'].values():
            try:
                singularities.append((planet['singularity'], planet['singularityString']))
            except KeyError:
                pass
            try:
                planet_types.append((planet['type'], planet['typeString']))
            except KeyError:
                pass
            planet_count += 1
    return (
        np.unique(planet_types, axis=0), 
        np.unique(singularities, axis=0), 
        np.unique(star_types, axis=0), 
        planet_count
    )


with mp.Pool(mp.cpu_count() - 1) as pool:
    planet_types, singularities, star_types, planet_count = zip(*pool.imap_unordered(partial(process_universe, map_strings), files, chunksize=100))
planet_types = np.unique(np.concatenate([x for x in planet_types if len(x) > 0]), axis=0)
singularities = np.unique(np.concatenate([x for x in singularities if len(x) > 0]), axis=0)
star_types = np.unique(np.concatenate([x for x in star_types if len(x) > 0]), axis=0)


time: 15.4 s (started: 2021-10-29 08:03:15 +01:00)


In [8]:
display(pd.DataFrame.from_records(planet_types, columns=['code', 'type']))
display(pd.DataFrame.from_records(star_types, columns=['code', 'type']))

,code,type
0,1,Lava
1,1,Volcanic Ash
2,2,Mediterranean
3,2,Oceanic Jungle
4,2,Prairie
5,2,Red Stone
6,2,Sakura Ocean
7,2,Waterworld
8,3,Arid Desert
9,3,Ashen Gelisol


,code,type
0,0,A type star
1,0,B type star
2,0,F type star
3,0,G type star
4,0,K type star
5,0,M type star
6,0,O type star
7,1,Blue giant
8,1,Red giant
9,1,White giant


time: 8.79 ms (started: 2021-10-29 08:03:31 +01:00)


In [9]:
OIL_SPEED_MULTIPLIER = 4e-5
DYSON_SPHERE_BUFFER = 1.35

def reorder(df):
    resource_cols = vein_types + gas_types
    first_cols = [x for x in df.columns if x not in resource_cols]
    df[resource_cols] = df[resource_cols].fillna(0)
    return df[first_cols + resource_cols]

def scan_systems(universe):
    home_star_id = universe['meta']['birthStarId']
    home_planet_id = universe['meta']['birthPlanetId']
    seed = universe['meta']['seed']
    mtime = universe['meta']['mtime']
    stars = []
    planets = []
    
    for star in universe['star'].values():
        max_dyson_rad = star['dysonRadius'] * 2

        star_dict = {
            'seed': seed,
            'star': star['name'],
            'home_system': star['id'] == home_star_id,
            'spectrum': spectra_map[star['spectr']],
            'type': star_type_map[star['type']],
            'luminosity': star['luminosity'],
        }
        stars.append(star_dict)

        for planet in star['planet'].values():
            planet_dict = {
                'seed': seed,
                'star': star['name'],
                'planet': planet['name'],
                'home_system': star['id'] == home_star_id,
                'homeworld': planet['id'] == home_planet_id,
                'type': planet.get('typeString', None),
                'tidal_lock': (planet['singularity'] & singularity_flags['TidalLocked']) > 0,
                'in_sphere': planet['sunDistance'] <= max_dyson_rad / DYSON_SPHERE_BUFFER,
                'ocean': planet.get('waterItem', None),
                'landPercent': planet.get('landPercent', 0.),
            }
            if 'vein' in planet:
                planet_veins = {vein_type_map[k]: v for k, v in planet['vein'].items()}
            elif 'gas' in planet:
                planet_veins = {(k + ' gas'): float(v['gasSpeed']) for k,v in planet['gas'].items()}
            else:
                raise ValueError('Bad planet: {}'.format(planet['name']))
            assert all(isinstance(x, (int, float)) for x in planet_veins.values()), planet_veins
            planets.append({**planet_dict, **planet_veins})
            
    planet_df = pd.DataFrame.from_records(planets)
    planet_df['Oil'] *= OIL_SPEED_MULTIPLIER
    
    star_df = pd.DataFrame.from_records(stars)
    systems = planet_df.groupby('star')
    star_resources = systems.sum().reindex(vein_types + gas_types, axis='columns')
    star_resources['planets'] = systems['planet'].count()
    oceans = (systems['ocean'].value_counts().unstack() > 0)[['Water', 'Sulfuric acid']]
    oceans.reindex(star_df['star'], fill_value=False)
    star_df = (star_df
               .merge(star_resources, left_on='star', right_index=True, validate='1:1')
               .merge(oceans.reindex(star_df['star'], fill_value=False), 
                      left_on='star', right_index=True, validate='1:1'))
    
    seed_df =  pd.DataFrame.from_records([{'seed': seed, 'ts': mtime}])
    
    return star_df, planet_df, seed_df

star_df, planet_df, seed_df = scan_systems(universe)
display(star_df.head())
display(planet_df.head())
display(seed_df.head())

,seed,star,home_system,spectrum,type,luminosity,Iron,Copper,Silicon,Titanium,...,Organic Crystal,Optical Grating,Spin. Stal. Crystal,Uni. Magnets,Hydrogen gas,Deuterium gas,Fire ice gas,planets,Water,Sulfuric acid
0,18238299,Procyon,True,G,Main Sequence,0.978408,23539812.0,33240354.0,13467962.0,13893383.0,...,0.0,0.0,0.0,0.0,0.250872,0.0,0.597154,4,True,False
1,18238299,Tramontana,False,M,Main Sequence,0.583942,14053173.0,28310788.0,3529581.0,37695658.0,...,0.0,2664609.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
2,18238299,Gamma Vulpeculae,False,K,Main Sequence,0.823254,36914259.0,41160402.0,20979566.0,37358159.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
3,18238299,YedPrior,False,M,Main Sequence,0.497194,26414756.0,18726224.0,19573996.0,39492924.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3,True,False
4,18238299,Lambda Sagittae,False,K,Main Sequence,0.817663,9531719.0,1975948.0,4960219.0,31078570.0,...,0.0,958814.0,0.0,0.0,0.000000,0.0,0.000000,2,True,False


,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,landPercent,...,Hydrogen gas,Oil,Fire ice,Fractal Silicon,Optical Grating,Spin. Stal. Crystal,Kimberlite,Organic Crystal,Deuterium gas,Uni. Magnets
0,18238299,Procyon,Procyon I,True,False,Lava,False,True,None,0.873394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18238299,Procyon,Procyon II,True,False,Gobi,False,False,None,0.989456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18238299,Procyon,Procyon III,True,False,Ice Giant,False,False,None,0.000000,...,0.250872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18238299,Procyon,Procyon IV,True,True,Mediterranean,False,False,Water,0.601668,...,NaN,53.22388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238299,Tramontana,Tramontana I,False,False,Arid Desert,False,True,None,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,seed,ts
0,18238299,2021-09-07 18:25:41.524348


time: 39.5 ms (started: 2021-10-29 08:03:31 +01:00)


In [10]:
with mp.Pool(mp.cpu_count() - 1) as pool:
    star_dfs, planet_dfs, seed_dfs = zip(*pool.imap_unordered(partial(process_universe, scan_systems), files, chunksize=100))
star_df = reorder(pd.concat(star_dfs))
planet_df = reorder(pd.concat(planet_dfs))
seed_df = pd.concat(seed_dfs)

time: 32.3 s (started: 2021-10-29 08:03:31 +01:00)


In [11]:
home_systems = star_df.query('home_system == True').copy()
home_systems.head()

,seed,star,home_system,spectrum,type,luminosity,planets,Water,Sulfuric acid,Iron,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spin. Stal. Crystal,Uni. Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,4708753,Muscida,True,G,Main Sequence,0.919383,4,True,False,10695920.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.770142,0.034026,0.000000
0,4719245,Beta Virginis,True,G,Main Sequence,0.992047,4,True,False,25924534.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.811467,0.035268,0.000000
0,4723975,Tegmen,True,G,Main Sequence,0.938546,4,True,False,25782118.0,...,13105608.0,0.0,0.0,0.0,0.0,0.0,0.0,0.802756,0.034657,0.000000
0,4724353,RanaSecunda,True,G,Main Sequence,1.086060,4,True,False,5047259.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.870748,0.035437,0.000000
0,4733744,Epsilon Scuti,True,G,Main Sequence,1.014201,4,True,False,7786572.0,...,5859296.0,0.0,0.0,0.0,0.0,0.0,0.0,0.245372,0.000000,0.581507


time: 18.7 ms (started: 2021-10-29 08:04:03 +01:00)


In [12]:
home_system_planets = planet_df.query('home_system == True').copy()
home_system_planets.head()

,seed,star,planet,home_system,homeworld,type,tidal_lock,in_sphere,ocean,landPercent,...,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spin. Stal. Crystal,Uni. Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
0,4708753,Muscida,Muscida I,True,False,None,False,True,None,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,4708753,Muscida,Muscida II,True,False,None,False,False,None,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.770142,0.034026,0.0
2,4708753,Muscida,Muscida III,True,True,None,False,False,Water,0.622287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,4708753,Muscida,Muscida IV,True,False,None,False,False,None,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
0,4719245,Beta Virginis,Beta Virginis I,True,False,Lava,False,True,None,0.859558,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


time: 24.3 ms (started: 2021-10-29 08:04:03 +01:00)


In [13]:
home_system_planets['TL|IS'] = home_system_planets['tidal_lock'] | home_system_planets['in_sphere']
home_system_planets['TL&IS'] = home_system_planets['tidal_lock'] & home_system_planets['in_sphere']
tl_is = home_system_planets.groupby(['seed', 'star'])[['TL|IS', 'TL&IS', 'tidal_lock', 'in_sphere']].sum()
pd.DataFrame({col: tl_is.reset_index().groupby(col)['star'].count()
              for col in tl_is.columns}).fillna(0).astype(int)

,TL|IS,TL&IS,tidal_lock,in_sphere
0,3832,9141,8922,3934
1,5400,211,427,5418
2,120,0,3,0


time: 27.6 ms (started: 2021-10-29 08:04:03 +01:00)


In [14]:
homeworlds = home_system_planets.query('homeworld == True')
landmass = homeworlds[['seed', 'landPercent']].rename(columns={'landPercent': 'land'})

time: 6.86 ms (started: 2021-10-29 08:04:03 +01:00)


In [15]:
res_df = star_df.copy()
res_df[vein_types + gas_types] = res_df[vein_types + gas_types] > 0
res_probs = res_df.groupby(['spectrum', 'type'])[ocean_types
                                                 + vein_types 
                                                 + gas_types
                                                ].mean()
res = 'Organic Crystal'
res_probs.loc[res_probs[res] > 0].sort_values(res, ascending=False).style.format(precision=2)

,,Water,Sulfuric acid,Iron,Copper,Silicon,Titanium,Stone,Coal,Oil,Fire ice,Kimberlite,Fractal Silicon,Organic Crystal,Optical Grating,Spin. Stal. Crystal,Uni. Magnets,Hydrogen gas,Deuterium gas,Fire ice gas
spectrum,type,,,,,,,,,,,,,,,,,,,
F,Main Sequence,0.79,0.39,1.00,1.00,1.00,1.00,1.00,1.00,0.54,0.69,0.31,0.21,0.44,0.18,0.26,0.00,0.46,0.25,0.21
G,Main Sequence,0.85,0.29,1.00,1.00,1.00,0.99,1.00,1.00,0.59,0.69,0.30,0.21,0.42,0.19,0.28,0.00,0.46,0.25,0.21
K,Main Sequence,0.85,0.18,0.99,0.99,0.98,0.96,0.99,0.99,0.50,0.79,0.31,0.27,0.41,0.23,0.31,0.00,0.31,0.17,0.14
M,Main Sequence,0.88,0.10,0.99,0.99,0.98,0.97,0.99,0.99,0.43,0.89,0.46,0.50,0.35,0.42,0.37,0.00,0.31,0.17,0.14
A,Main Sequence,0.54,0.52,1.00,1.00,1.00,1.00,1.00,1.00,0.29,0.56,0.50,0.33,0.23,0.30,0.17,0.00,0.53,0.30,0.25
B,Main Sequence,0.43,0.71,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.61,0.32,0.20,0.09,0.18,0.04,0.00,0.65,0.39,0.32
O,Main Sequence,0.39,0.83,1.00,1.00,1.00,1.00,1.00,1.00,0.11,0.71,0.79,0.59,0.09,0.57,0.08,0.00,0.68,0.42,0.35
M,Giant,0.01,0.37,1.00,1.00,0.94,0.94,1.00,1.00,0.01,0.00,0.46,0.29,0.01,0.24,0.01,0.00,0.55,0.30,0.25


time: 247 ms (started: 2021-10-29 08:04:03 +01:00)


In [16]:
def S_scaler(f):
    return '{:.2f}/s'.format(f)

def M_scaler(f):
    if f < 10e6:
        return '{:.1f}M'.format(f / 1e6)
    else:
        return '{:.0f}M'.format(f / 1e6)

def f2_scaler(f):
    return '{:.2f}'.format(f)
    
shaders = {
    'limit': M_scaler,
    'land': f2_scaler,
    **{col: S_scaler for col in S_scaled},
    **{col: M_scaler for col in M_scaled},
    **{col + ' (Start)': S_scaler for col in S_scaled},
    **{col + ' (Start)': M_scaler for col in M_scaled},
}
formatters = {**shaders,
    'ts': lambda x: x.strftime('%b-%d')
}

def shader(series, pct):
    pct_subset = pct.loc[series.index, series.name] ** (4/5)
    cmap = mpl.cm.get_cmap('coolwarm_r')
    rgb = [sns.set_hls_values(color=cmap(x), h=None, l=0.75, s=None) for x in pct_subset]
    return ['background: {};'.format(mpl.colors.rgb2hex(x)) for x in rgb]

def show(pct, df):
    shade_cols = [x for x in df.columns if x in shaders.keys()]
    display(HTML(
        df.style
        .apply(func=shader, pct=pct[shade_cols], axis='index', subset=shade_cols)
        .format(formatter=formatters).to_html()
    ))

time: 806 µs (started: 2021-10-29 08:04:03 +01:00)


In [17]:
home_resources = home_systems.set_index('seed')[base_types + ['Oil', 'Fire ice', 'Fire ice gas']]
home_resources['limit'] =  home_resources[base_types].min(axis='columns')

seed_res = (
    tl_is.reset_index(level='star')
    .merge(seed_df, on='seed', validate='1:1')
    .merge(landmass, on='seed', validate='1:1')
    .merge(home_resources, on='seed', validate='1:1')
    .merge(
        star_df.groupby('seed')[ocean_types + vein_types + gas_types].sum(), 
        on='seed', validate='1:1', suffixes=[' (Start)', ''])
).set_index('seed')

time: 61.6 ms (started: 2021-10-29 08:04:03 +01:00)


In [19]:
recent = dt.now() - td(days=7)

potential = (
    seed_res
    .query('(`Fire ice (Start)` > 1e6) | ((`Fire ice (Start)` > 0) & (`Fire ice gas (Start)` > 0))')
    # .query('`ts` > @recent')
    .query('`TL|IS` >= 1')
    # .query('`tidal_lock` >= 1')
    # .query('limit > 6e6')
    # .query('`Organic Crystal` > 50e6')
    # .query('`land` > .6')
    
    # .sort_values('limit', ascending=False)
    # .sort_values('Organic Crystal', ascending=False)
)

rank_pct = seed_res.rank(pct=True, method='min')
# rank_pct = potential.rank(pct=True, method='min')

rsrc_cols = vein_types + gas_types
# rsrc_cols = [x for x in potential_pct.columns if '(Start)' in x]

# sort_idx = rank_pct['limit'].sort_values(ascending=False).index
# sort_idx = rank_pct[rsrc_cols].quantile(0.1, axis='columns').sort_values(ascending=False).index
sort_idx = rank_pct[rsrc_cols].mean(axis='columns').sort_values(ascending=False).index

potential = potential.loc[sort_idx]
pre_run = potential.index.isin(previous_runs)
show(rank_pct, potential.loc[pre_run])
show(rank_pct, potential.loc[~pre_run].head(25))


KeyError: '[73521895, 70809808, 34033620, 28588019, 1847970, 82278905, 84140038, 19409367, 80089882, 25379415, 72112195, 86114713, 43805604, 98808195, 94520639, 38374847, 28751893, 19988791, 57087785, 99643775, 55668057, 14013365, 7829149, 4875541, 96104142, 74378969, 35291555, 39721766, 47864258, 47781395, 71768632, 9935240, 82499493, 61249120, 70604465, 14842470, 4239745, 23499157, 28878758, 16019900, 80647483, 91892069, 62255726, 96983696, 20411128, 46271028, 95593191, 48045604, 21474451, 47330681, 16790827, 92295032, 28108879, 60277916, 23488080, 97220008, 23492734, 27553193, 64801147, 67150234, 95363235, 51219620, 74208812, 73123498, 65270325, 56464489, 67409719, 14875327, 12930206, 87843786, 63906475, 66017300, 92844421, 3015483, 63191737, 2281680, 57749571, 7914977, 86165905, 10139785, 12273396, 12401650, 99210878, 76065942, 12587695, 51517280, 72987632, 78912762, 52054571, 61013337, 6974237, 20022884, 26539488, 58881704, 80429718, 62473366, 65856211, 23739139, 72300426, 5562283

time: 168 ms (started: 2021-10-29 08:05:01 +01:00)


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=[10,10], sharex=True, sharey=True)
(seed_res[[x for x in seed_res.columns if 'Start' in x and 'Oil' not in x and 'gas' not in x]] / 1e6).plot.kde(ax=axs[0])
(potential[[x for x in potential.columns if 'Start' in x and 'Oil' not in x and 'gas' not in x]] / 1e6).plot.kde(ax=axs[1])
plt.xlim(0, 40)
plt.ylim(0, .4)


In [ ]:
seed_res['Oil (Start)'].plot.kde()
potential['Oil (Start)'].plot.kde()


In [ ]:
seed_res.query('`Fire ice gas (Start)` > 0')['Fire ice gas (Start)'].plot.kde()
potential.query('`Fire ice gas (Start)` > 0')['Fire ice gas (Start)'].plot.kde()

In [ ]:
cmap = mpl.cm.get_cmap('coolwarm_r')

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, power_list):
    nrows = len(power_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(cmap_category + ' colormaps', fontsize=14)

    for ax, power in zip(axs, power_list):
        ax.imshow(gradient ** power, aspect='auto', cmap=cmap)
        ax.text(-.01, .5, '{:.3f}'.format(power), va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

power_list = [1, 4/5, 3/4, 2/3, 1/2]
plot_color_gradients('Powers', power_list)
